In [2]:
import pandas as pd
import numpy as np

# Create the full data table

In [3]:
data_sets = ["data/parsed/2017-10-11.csv.gz",
            "data/parsed/2017-10-12.csv.gz",
            "data/parsed/2017-10-13.csv.gz",
            "data/parsed/2017-10-14.csv.gz",
            "data/parsed/2017-10-15.csv.gz",
            "data/parsed/2017-10-16.csv.gz",
            "data/parsed/2017-10-17.csv.gz",
            "data/parsed/2017-10-18.csv.gz",
            "data/parsed/2017-10-19.csv.gz",
            "data/parsed/2017-10-20.csv.gz",
            "data/parsed/2017-10-21.csv.gz",
            "data/parsed/2017-10-22.csv.gz",
            "data/parsed/2017-10-23.csv.gz"]

full_data = pd.DataFrame()

for file in data_sets:
    partial_data = pd.read_csv(file)
    full_data = pd.concat([full_data, partial_data], axis=0)
    
full_data.shape

(11440778, 19)

# Remove unnecessary columns

In [12]:
#drop the obviously useless columns
full_data = full_data.drop(['dir'], axis=1)
full_data = full_data.drop(['tst'], axis=1)
full_data = full_data.drop(['hdg'], axis=1)
full_data = full_data.drop(['odo'], axis=1)
full_data = full_data.drop(['oday'], axis=1)
full_data = full_data.drop(['stop_index'], axis=1)
full_data = full_data.drop(['source'], axis=1)
full_data = full_data.drop(['jrn'], axis=1)
full_data = full_data.drop(['line'], axis=1)
full_data = full_data.drop(['oper'], axis=1)
full_data = full_data.drop(['start'], axis=1)
full_data = full_data.drop(['timestamp'], axis=1)

In [18]:
# Give columns more meaningful labels
full_data.rename(columns = {'desi':'line'}, inplace = True)
full_data.rename(columns = {'dl':'delay'}, inplace = True)
full_data.rename(columns = {'spd':'speed'}, inplace = True)
full_data.rename(columns = {'tsi':'time'}, inplace = True)
full_data.rename(columns = {'veh':'vehicle'}, inplace = True)
full_data.rename(columns = {'lat':'latitude'}, inplace = True)
full_data.rename(columns = {'long':'longitude'}, inplace = True)

In [22]:
full_data.head()

,line,delay,latitude,longitude,speed,time,vehicle
0,560,120,60.250462,25.009306,0.09,1507754213,0012_01517
1,23,-33,60.202652,24.903437,10.03,1507754213,0012_01612
2,23,39,60.202940,24.907900,0.00,1507754199,02f42add
3,571,50,60.290020,25.006930,0.00,1507754210,037bee32
4,55,92,60.192116,24.978412,0.00,1507754132,046524ec


# Weather data

In [4]:
weather = pd.read_csv("data/all_weather.csv")
weather.head()

,apparentTemperature,cloudCover,dewPoint,humidity,icon,ozone,precipIntensity,precipProbability,precipType,pressure,summary,temperature,time,uvIndex,visibility,windBearing,windGust,windSpeed
0,5.81,0.63,7.21,0.96,partly-cloudy-night,314.18,0.0076,0.30,rain,993.96,Mostly Cloudy,7.77,1507680000,0,9.72,68,11.99,10.93
1,5.98,0.70,7.19,0.96,partly-cloudy-night,314.63,0.0076,0.27,rain,993.88,Mostly Cloudy,7.78,1507681200,0,9.24,65,10.77,10.06
2,6.16,0.76,7.18,0.96,partly-cloudy-night,315.07,0.0051,0.24,rain,993.80,Mostly Cloudy,7.78,1507682400,0,8.74,62,9.53,9.22
3,6.43,0.82,7.18,0.96,partly-cloudy-night,315.52,0.0076,0.30,rain,993.75,Mostly Cloudy,7.77,1507683600,0,8.69,55,8.30,7.89
4,6.28,0.83,7.13,0.96,partly-cloudy-night,315.98,0.0051,0.27,rain,993.75,Mostly Cloudy,7.72,1507684800,0,8.64,45,7.77,8.29


# Combine weather data

In [62]:
full_data = full_data.sort_values('time')

full_data = pd.merge_asof(full_data, weather, on='time')

In [64]:
full_data[20:30]

,line,delay,latitude,longitude,speed,time,vehicle,apparentTemperature,cloudCover,dewPoint,...,precipProbability,precipType,pressure,summary,temperature,uvIndex,visibility,windBearing,windGust,windSpeed
20,562N,50,60.238370,25.109220,0.0,1507680170,f9e9209c,5.81,0.63,7.21,...,0.3,rain,993.96,Mostly Cloudy,7.77,0.0,9.72,68.0,11.99,10.93
21,1B,P0Y0M0DT9H40M48.000S,61.479612,23.795426,NaN,1507680174,Paunu_141,5.81,0.63,7.21,...,0.3,rain,993.96,Mostly Cloudy,7.77,0.0,9.72,68.0,11.99,10.93
22,1B,P0Y0M0DT9H40M48.000S,61.479612,23.795426,NaN,1507680174,Paunu_141,5.81,0.63,7.21,...,0.3,rain,993.96,Mostly Cloudy,7.77,0.0,9.72,68.0,11.99,10.93
23,235N,120,60.200470,24.687620,0.0,1507680180,76d53e3e,5.81,0.63,7.21,...,0.3,rain,993.96,Mostly Cloudy,7.77,0.0,9.72,68.0,11.99,10.93
24,615,10,60.307820,24.960752,0.0,1507680190,4d63ce12,5.81,0.63,7.21,...,0.3,rain,993.96,Mostly Cloudy,7.77,0.0,9.72,68.0,11.99,10.93
25,562N,35,60.242165,25.108975,0.0,1507680215,f9e9209c,5.81,0.63,7.21,...,0.3,rain,993.96,Mostly Cloudy,7.77,0.0,9.72,68.0,11.99,10.93
26,235N,115,60.201458,24.697430,0.0,1507680235,76d53e3e,5.81,0.63,7.21,...,0.3,rain,993.96,Mostly Cloudy,7.77,0.0,9.72,68.0,11.99,10.93
27,235N,115,60.201458,24.697430,0.0,1507680235,76d53e3e,5.81,0.63,7.21,...,0.3,rain,993.96,Mostly Cloudy,7.77,0.0,9.72,68.0,11.99,10.93
28,1B,P0Y0M0DT9H42M9.000S,61.479612,23.795426,NaN,1507680255,Paunu_141,5.81,0.63,7.21,...,0.3,rain,993.96,Mostly Cloudy,7.77,0.0,9.72,68.0,11.99,10.93
29,615,31,60.302518,24.955252,0.0,1507680271,4d63ce12,5.81,0.63,7.21,...,0.3,rain,993.96,Mostly Cloudy,7.77,0.0,9.72,68.0,11.99,10.93


# Cull the data to only observe certain routes

In [75]:
unique_lines = pd.unique(full_data.line.ravel())

In [76]:
observed_lines = ['550', '731', '55', '83', '94N']

culled_data = full_data.loc[full_data['line'].isin(observed_lines)]

In [78]:
culled_data.shape

(358210, 24)

In [79]:
culled_data.head()

,line,delay,latitude,longitude,speed,time,vehicle,apparentTemperature,cloudCover,dewPoint,...,precipProbability,precipType,pressure,summary,temperature,uvIndex,visibility,windBearing,windGust,windSpeed
992,550,50,60.16880,24.80383,0.0,1507685150,25cb9fcf,6.28,0.83,7.13,...,0.27,rain,993.75,Mostly Cloudy,7.72,0.0,8.64,45.0,7.77,8.29
993,550,50,60.16880,24.80383,0.0,1507685150,25cb9fcf,6.28,0.83,7.13,...,0.27,rain,993.75,Mostly Cloudy,7.72,0.0,8.64,45.0,7.77,8.29
1008,550,80,60.20982,25.07738,0.0,1507685180,55a3d4a3,6.28,0.83,7.13,...,0.27,rain,993.75,Mostly Cloudy,7.72,0.0,8.64,45.0,7.77,8.29
1009,550,80,60.20982,25.07738,0.0,1507685180,55a3d4a3,6.28,0.83,7.13,...,0.27,rain,993.75,Mostly Cloudy,7.72,0.0,8.64,45.0,7.77,8.29
1028,550,67,60.17083,24.81079,0.0,1507685227,25cb9fcf,6.28,0.83,7.13,...,0.27,rain,993.75,Mostly Cloudy,7.72,0.0,8.64,45.0,7.77,8.29


# Handle missing values

In [8]:
# TODO

# Export the final dataset

In [80]:
culled_data.to_csv('culled_data.csv.gz', index=False, compression='gzip')